#### Install OpenAI using Pip

In [10]:
!python3 -m pip install openai
!python3 -m pip install sounddevice
!python3 -m pip install numpy
!python3 -m pip install pandas


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/9e/5f/b0ed836c904bda20804f496355febdd258acc2ceb6e74809704e5fb3942e/pandas-2.1.3-cp39-cp39-mac

#### Get API Key

In [24]:
import json

with open("config.json", 'r') as config_file:
    config = json.load(config_file)

api_key = config.get("API_KEY")


In [25]:
import openai
import os
import pandas as pd
import uuid
import time
import csv

openai.api_key = api_key
audio_download_path = "../youtube_to_audio/audio_download/"
audio_files = [f for f in os.listdir(audio_download_path) if os.path.isfile(os.path.join(audio_download_path, f))]


In [21]:
# transcripts_df = pd.DataFrame(columns=['id', 'lecture', 'Notes'])
lecture_transcript = []

for audio_file_name in audio_files:
    temp_row_entry = {}
    audio_file = open(f"{audio_download_path}{audio_file_name}", "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    print(f'Transcribed file {audio_file_name}...')
    random_id = str(uuid.uuid4())
    temp_row_entry["Id"] = random_id
    temp_row_entry["lecture"] = transcript.text
    lecture_transcript.append(temp_row_entry)
    time.sleep(2)


Transcribed file lecture1.mp3...
Transcribed file lecture11.mp3...
Transcribed file lecture10.mp3...
Transcribed file lecture2.mp3...
Transcribed file lecture12.mp3...
Transcribed file lecture13.mp3...
Transcribed file lecture3.mp3...
Transcribed file lecture7.mp3...
Transcribed file lecture17.mp3...
Transcribed file lecture16.mp3...
Transcribed file lecture6.mp3...
Transcribed file lecture4.mp3...
Transcribed file lecture14.mp3...
Transcribed file lecture28.mp3...
Transcribed file lecture29.mp3...
Transcribed file lecture15.mp3...
Transcribed file lecture5.mp3...
Transcribed file lecture8.mp3...
Transcribed file lecture18.mp3...
Transcribed file lecture24.mp3...
Transcribed file lecture30.mp3...
Transcribed file lecture25.mp3...
Transcribed file lecture19.mp3...
Transcribed file lecture9.mp3...
Transcribed file lecture27.mp3...
Transcribed file lecture26.mp3...
Transcribed file lecture22.mp3...
Transcribed file lecture23.mp3...
Transcribed file lecture21.mp3...
Transcribed file lectur

In [22]:
csv_columns = ['Id', 'lecture']
# Save the DataFrame to a CSV file
transcripts_csv_path = 'assets/transcripts.csv'
with open(transcripts_csv_path, 'w') as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()
    for data in lecture_transcript:
        writer.writerow(data)